In [1]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit.ignis
!pip install qiskit_machine_learning

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [16]:
from qiskit import *
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
#from qiskit import execute
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.primitives import BackendEstimator
from qiskit import Aer

import torch
from qiskit.circuit import ParameterVector
from torch import Tensor
import torch.nn as nn
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from tqdm import tqdm

from torch.utils.data import DataLoader, Dataset
import torch.utils.data as data_utils

from collections import Counter
import math

In [3]:
from pathlib import Path
import pandas as pd
from joblib import load, dump

# Load Dataset

In [4]:
!git clone https://github.com/CQCL/qnlp_lorenz_etal_2021_resources
!mv qnlp_lorenz_etal_2021_resources/datasets mc_rp_dataset

Cloning into 'qnlp_lorenz_etal_2021_resources'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 90 (delta 40), reused 57 (delta 25), pack-reused 0
Unpacking objects: 100% (90/90), 56.07 KiB | 883.00 KiB/s, done.


In [5]:
mc_rp_sets_path = Path("mc_rp_dataset")
mc_rp_sets = list(mc_rp_sets_path.glob("*.txt"))
mc_datasets, rp_datasets = list(filter(lambda x: x.name.startswith("mc"), mc_rp_sets)), list(filter(lambda x: x.name.startswith("rp"), mc_rp_sets))

In [11]:
# Reading MC and RP data and creating data loaders from our data loading strategy
def read_process_mcrp(datapaths: dict):
  def rm(text):
    return " ".join(list(map(lambda x: x[:x.find('_')], text.split())))
  retval = {}
  for datapath in datapaths:
    if "rp" in str(datapath):
      sel = 2
    else:
      sel = 3
    df = pd.DataFrame(list(map(lambda x: [int(x[0]), x[sel:]], datapath.read_text().split("\n"))), columns=['label', 'text'])
    df['text'] = df['text'].apply(rm)
    retval[datapath.name.split(".")[0]] = df
  return retval

mc_data, rp_data = read_process_mcrp(mc_datasets), read_process_mcrp(rp_datasets)
MRP_BATCH_SIZE = 30

In [12]:
def data_frame_simple_txt2vec(data,keystr,min_length=6,w_dict=None):
    txt_lst=[]
    max_len=0
    for txt in data[keystr].text:
        txtsplit=txt.split()
        txt_lst=txt_lst+txtsplit
        if max_len< len(txtsplit):
            max_len=len(txtsplit)
    w_set=set(txt_lst)

    comm_lst=Counter(txt_lst).most_common()

    if w_dict==None:
        w_dict={}
        ind=len(comm_lst)
        for elem in comm_lst:
            w_dict[elem[0]]=ind
            ind-=1
    #w_dict={}
    #ind=1
    #for elem in w_set:
    #    w_dict[elem]=ind
    #    ind+=1

    if max_len%2 !=0:
        max_len+=1
    if max_len<min_length:
        max_len=min_length

    label_list=[]
    Txt_list=[]
    for i, txt in enumerate(data[keystr].text):
        w_list=[]
        for word in data[keystr].text[i].split():
            if word in w_dict:
                w_list.append(w_dict[word])
            else:
                w_list.append(len(w_dict.keys())+1)
        if len(w_list) < max_len:
            w_list=w_list+(max_len-len(w_list))*[0]
        #print(torch.tensor(w_list))
        label_list.append(torch.tensor(data[keystr].label[i]))
        Txt_list.append(torch.tensor(w_list))
    label_tensor=torch.tensor(label_list)
    txt_tensor=torch.stack(Txt_list)

    return data_utils.TensorDataset(txt_tensor,label_tensor),w_dict

# Model

In [15]:
class PositionalEncoder(nn.Module):
    def __init__(self, embed_dim, max_seq_len=512):
        super().__init__()
        self.embed_dim = embed_dim

        # create constant 'pe' matrix with values dependant on pos and i
        pe = torch.zeros(max_seq_len, embed_dim)
        for pos in range(max_seq_len):
            for i in range(0, embed_dim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/embed_dim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/embed_dim)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.embed_dim)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + self.pe[:,:seq_len] #torch.autograd.Variable(self.pe[:,:seq_len], requires_grad=False)  # .cuda()
        return x

In [18]:
import random
class QSAL_qiskit(torch.nn.Module):
    def __init__(self,S,n,Denc,D):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.seq_num=S
        self.init_params_Q=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_K=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_V=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.params_input=[ParameterVector('IN'+str(i),n*(Denc+2)) for i in range(self.seq_num)]
        self.params_Q=[ParameterVector('Q'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.params_K=[ParameterVector('K'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.params_V=[ParameterVector('V'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.num_q=n
        self.Denc=Denc
        self.D=D
        self.d=n*(Denc+2)
        self.bit_string_Z=SparsePauliOp.from_list([('I'*(self.num_q-1)+'Z', 1)])
        self.pauli_strings=[]
        for i in range(self.d):
            string=['I']*self.num_q
            while string==['I']*self.num_q:
                for j in range(self.num_q):
                    a=random.randint(0, 4)
                    if a==0:
                        continue
                    elif a==1:
                        string[j]='X'
                    elif a==2:
                        string[j]='Y'
                    elif a==3:
                        string[j]='Z'
            self.pauli_strings.append(SparsePauliOp.from_list([("".join(string), 1)]))

        Q_qnn=[EstimatorQNN(circuit=self.QSAL_cir("Q",i),observables=[self.bit_string_Z], input_params=self.params_input[i], weight_params=self.params_Q[i]) for i in range(self.seq_num)]
        K_qnn=[EstimatorQNN(circuit=self.QSAL_cir("K",i),observables=[self.bit_string_Z], input_params=self.params_input[i], weight_params=self.params_K[i]) for i in range(self.seq_num)]
        V_qnn=[EstimatorQNN(circuit=self.QSAL_cir("V",i),observables=self.pauli_strings, input_params=self.params_input[i], weight_params=self.params_V[i]) for i in range(self.seq_num)]

        self.Q_models=[TorchConnector(Q_qnn[i], initial_weights=self.init_params_Q[i]) for i in range(self.seq_num)]
        self.K_models=[TorchConnector(K_qnn[i], initial_weights=self.init_params_K[i]) for i in range(self.seq_num)]
        self.V_models=[TorchConnector(V_qnn[i], initial_weights=self.init_params_V[i]) for i in range(self.seq_num)]

    def QSAL_cir(self,type,indx):

        qc=QuantumCircuit(self.num_q)
        if type=="Q":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_Q[indx])

        elif type=="K":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_K[indx])

        elif type=="V":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_V[indx])

        return qc

    def Feature_map(self,qc,params):
        indx=0
        for j in range(self.num_q):
            qc.rx(params[indx],j)
            qc.ry(params[indx+1],j)
            indx+=2
        for i in range(self.Denc):
            for j in range(self.num_q):
                qc.cx(j,(j+1)%self.num_q)

            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qc.ry(params[indx],j)
                indx+=1


    def ansatz(self,qc,params):
        indx=0
        for j in range(self.num_q):
            qc.rx(params[indx],j)
            qc.ry(params[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qc.cx(j,(j+1)%self.num_q)

            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qc.ry(params[indx],j)
                indx+=1

    def forward(self,input):

        Q_output=torch.stack([self.Q_models[i](input[:,i]) for i in range(self.seq_num)])
        K_output=torch.stack([self.K_models[i](input[:,i]) for i in range(self.seq_num)])
        V_output=torch.stack([self.V_models[i](input[:,i]) for i in range(self.seq_num)])
        batch_size=len(input)
        Q_output=Q_output.transpose(0,2).repeat((self.seq_num,1,1))
        K_output=K_output.transpose(0,2).repeat((self.seq_num,1,1)).transpose(0,2)
        #print(V_output.size())
        #Q_grid, K_grid=torch.meshgrid(Q_output, K_output, indexing='ij')
        alpha=torch.exp(-(Q_output-K_output)**2)
        alpha=alpha.transpose(0,1)
        V_output=V_output.transpose(0,1)
        output=[]

        for i in range(self.seq_num):

            Sum_a=torch.sum(alpha[:,i,:],-1)
            div_sum_a=(1/Sum_a).repeat(self.d,self.seq_num,1).transpose(0,2)

            Sum_w=torch.sum(alpha[:,:,i].repeat((self.d,1,1)).transpose(0,2).transpose(0,1)*V_output*div_sum_a,1)
            output.append(Sum_w)
        return input+torch.stack(output).transpose(0,1)


class QSANN_qiskit(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.qsal_lst=[QSAL_qiskit(S,n,Denc,D) for _ in range(num_layers)]
        self.qnn=nn.Sequential(*self.qsal_lst)

    def forward(self,input):

        return self.qnn(input)

class QSANN_text_classifier(torch.nn.Module):
    def __init__(self,embed_dim,S,n,Denc,D,num_layers,v_cab_size=20):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.embed_dim=embed_dim
        self.num_seq=S
        self.num_q=n
        self.Denc=Denc
        self.q_embedding=self.num_q*(self.Denc+2)
        self.token_embedding=nn.Embedding(v_cab_size,embed_dim)#,_freeze=True)
        self.PE=PositionalEncoder(embed_dim)
        self.Qnn=QSANN_qiskit(S,n,Denc,D,num_layers)
        self.final_layer=nn.Linear(n*(Denc+2)*S, 1)
        self.final_layer=self.final_layer.float()

    def forward(self,input):
        token=self.token_embedding(input)
        x=self.PE(token)
        x=x*np.pi/x.norm()
        batch_size=x.size(0)
        x=torch.reshape(x, (batch_size,self.num_seq, self.q_embedding))
        x=self.Qnn(x)
        x=torch.flatten(x,start_dim=1)

        return torch.sigmoid(self.final_layer(x))

# Train Real Data

In [9]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

## Prepare mc datasets

In [52]:
mc_df_train,train_dict=data_frame_simple_txt2vec(mc_data,'mc_train_data')
mc_df_test,_=data_frame_simple_txt2vec(mc_data,'mc_test_data',w_dict=train_dict)

mc_trainloader = DataLoader(mc_df_train, shuffle=True, batch_size=MRP_BATCH_SIZE)
mc_testloader = DataLoader(mc_df_test, shuffle=True, batch_size=30)

In [53]:
model=QSANN_text_classifier(2,2,2,1,1,1)

/usr/local/lib/python3.10/dist-packages/qiskit_machine_learning/connectors/torch_connector.py:312: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._weights.data = torch.tensor(initial_weights, dtype=torch.float)


In [54]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params) # To count # of trainable parameters

89


In [40]:
n_epochs=180
optimizer = torch.optim.Adam(lr=0.005, params=model.parameters())
criterion = nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()
pbar = tqdm(total=len(mc_trainloader), leave=True)
if torch.cuda.is_available():
    devname = "cuda"
else:
    devname = "cpu"
device = torch.device(devname)
model.train()
model.to(device)

max_val=0

Test_acc=[]
Train_acc=[]
file_name="Feat_eco_model_222111_vocab_size20_mc"
pbar = tqdm(total=len(mc_trainloader), leave=True)
for epoch in range(n_epochs):
    acc=0
    for batch, (feature, label) in enumerate(mc_trainloader):
        optimizer.zero_grad()
        feature, label = feature.to(device), label.to(device)
        preds = model(feature).squeeze(1)
        loss = criterion(preds, label.float())
        preds = preds.ravel()
        label = label.ravel()
        acc += binary_accuracy(preds, label)#(preds == label).sum() / len(preds)
        #print(acc)
        #print(loss)

        loss.backward()
        optimizer.step()
        pbar.update()
        pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss.cpu().detach()}"
        pbar.refresh()
    acc=acc/(batch+1)
    print('Epoch',epoch,'Training Accuracy:', acc)
    feature, label=next(iter(mc_testloader))
    preds = model(feature).squeeze(1)
    loss = criterion(preds, label.float())
    preds = preds.ravel()
    label = label.ravel()
    acc1 = binary_accuracy(preds, label)
    print('Testing Accuracy:', acc1)
    Test_acc.append(acc1)
    Train_acc.append(acc)
    if max_val< (acc1+acc):
        max_val=acc1+acc
        torch.save(model.state_dict(),file_name)
pbar.close()

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:12<00:25, 12.52s/it]
Epoch: 0 | Batch: 0 | Loss 0.7063422799110413:  33%|███▎      | 1/3 [00:12<00:25, 12.52s/it]
Epoch: 0 | Batch: 0 | Loss 0.7063422799110413:  67%|██████▋   | 2/3 [00:24<00:12, 12.34s/it]
Epoch: 0 | Batch: 1 | Loss 0.6839965581893921:  67%|██████▋   | 2/3 [00:24<00:12, 12.34s/it]
Epoch: 0 | Batch: 1 | Loss 0.6839965581893921: 100%|██████████| 3/3 [00:28<00:00,  8.31s/it]
Epoch: 0 | Batch: 2 | Loss 0.7035319805145264: 100%|██████████| 3/3 [00:28<00:00,  8.31s/it]

Epoch 0 Training Accuracy: tensor(0.4889)
Testing Accuracy: tensor(0.4000)



Epoch: 0 | Batch: 2 | Loss 0.7035319805145264: 4it [00:41, 10.27s/it]                       
Epoch: 1 | Batch: 0 | Loss 0.6968613266944885: 4it [00:41, 10.27s/it]
Epoch: 1 | Batch: 0 | Loss 0.6968613266944885: 5it [00:53, 10.99s/it]
Epoch: 1 | Batch: 1 | Loss 0.6711750030517578: 5it [00:53, 10.99s/it]
Epoch: 1 | Batch: 1 | Loss 0.6711750030517578: 6it [00:57,  8.45s/it]
Epoch: 1 | Batch: 2 | Loss 0.7244521379470825: 6it [00:57,  8.45s/it]

Epoch 1 Training Accuracy: tensor(0.5111)
Testing Accuracy: tensor(0.4333)



Epoch: 1 | Batch: 2 | Loss 0.7244521379470825: 7it [01:10, 10.04s/it]
Epoch: 2 | Batch: 0 | Loss 0.6845039129257202: 7it [01:10, 10.04s/it]
Epoch: 2 | Batch: 0 | Loss 0.6845039129257202: 8it [01:23, 10.84s/it]
Epoch: 2 | Batch: 1 | Loss 0.6937592625617981: 8it [01:23, 10.84s/it]
Epoch: 2 | Batch: 1 | Loss 0.6937592625617981: 9it [01:26,  8.54s/it]
Epoch: 2 | Batch: 2 | Loss 0.6761029958724976: 9it [01:26,  8.54s/it]

...

Epoch 178 Training Accuracy: tensor(1.)
Testing Accuracy: tensor(1.)



Epoch: 178 | Batch: 2 | Loss 0.019116448238492012: 538it [1:31:06,  9.83s/it]
Epoch: 179 | Batch: 0 | Loss 0.07795537263154984: 538it [1:31:06,  9.83s/it] 
Epoch: 179 | Batch: 0 | Loss 0.07795537263154984: 539it [1:31:18, 10.34s/it]
Epoch: 179 | Batch: 1 | Loss 0.07742279022932053: 539it [1:31:18, 10.34s/it]
Epoch: 179 | Batch: 1 | Loss 0.07742279022932053: 540it [1:31:23,  8.78s/it]
Epoch: 179 | Batch: 2 | Loss 0.011350708082318306: 540it [1:31:23,  8.78s/it]

Epoch 179 Training Accuracy: tensor(1.)


Epoch: 179 | Batch: 2 | Loss 0.011350708082318306: 540it [1:31:24, 10.16s/it]

Testing Accuracy: tensor(1.)


In [41]:
preds = []
labels = []
pbar = tqdm(total=len(mc_testloader), leave=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
for batch, (feature, label) in enumerate(mc_testloader):
  feature, label = feature.to(device), label.to(device)
  with torch.no_grad():
    predictions = model(feature)
    preds.append(predictions.cpu().numpy())
    labels.append(label.cpu().numpy())
  pbar.update()
  pbar.desc = f"Batch: {batch}"
  pbar.refresh()
pbar.close()
preds = torch.tensor(preds).ravel()
# preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
labels = torch.tensor(labels).ravel()
#acc = (preds == labels).sum() / len(preds)
acc =binary_accuracy(preds, labels)
print(acc)

Batch: 0: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]

tensor(1.)



<ipython-input-41-5ea85aa1f8fc>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  preds = torch.tensor(preds).ravel()


## RP task

In [55]:
rp_df_train,train_dict=data_frame_simple_txt2vec(rp_data,'rp_train_data')
rp_df_test,_=data_frame_simple_txt2vec(rp_data,'rp_test_data',w_dict=train_dict)

rp_trainloader = DataLoader(rp_df_train, shuffle=True, batch_size=20)
rp_testloader = DataLoader(rp_df_test, shuffle=True, batch_size=31)

In [56]:
model_2=QSANN_text_classifier(2,2,2,1,2,1,v_cab_size=100)

In [50]:
trainable_params = sum(p.numel() for p in model_2.parameters() if p.requires_grad)
print(trainable_params)

261


In [57]:
n_epochs=300
optimizer = torch.optim.Adam(lr=0.005, params=model_2.parameters())
criterion = nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()

pbar = tqdm(total=len(rp_trainloader), leave=True)
if torch.cuda.is_available():
    devname = "cuda"
else:
    devname = "cpu"
device = torch.device(devname)
model_2.train()
model_2.to(device)
pbar = tqdm(total=len(rp_trainloader), leave=True)

file_name="Feat_eco_model_222121_vocab_size100_rp"
max_val=0
for epoch in range(n_epochs):
    acc=0
    for batch, (feature, label) in enumerate(rp_trainloader):
        optimizer.zero_grad()
        feature, label = feature.to(device), label.to(device)
        preds = model_2(feature).squeeze(1)
        loss = criterion(preds, label.float())
        preds = preds.ravel()
        label = label.ravel()
        acc += binary_accuracy(preds, label)

        loss.backward()
        optimizer.step()
        pbar.update()
        pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss.cpu().detach()}"
        pbar.refresh()
    acc=acc/(batch+1)
    print('Epoch',epoch,'Training Accuracy:', acc)
    feature, label=next(iter(rp_testloader))
    preds = model_2(feature).squeeze(1)
    loss = criterion(preds, label.float())
    preds = preds.ravel()
    label = label.ravel()
    acc1 = binary_accuracy(preds, label)
    print('Testing Accuracy:', acc1)
    if max_val< (acc1+acc):
        max_val=acc1+acc
        torch.save(model_2.state_dict(),file_name)
pbar.close()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]
Epoch: 188 | Batch: 2 | Loss 0.06955476105213165: 755it [2:38:03, 12.56s/it]


 25%|██▌       | 1/4 [00:12<00:38, 12.69s/it]

Epoch: 0 | Batch: 0 | Loss 0.6306712031364441:  25%|██▌       | 1/4 [00:12<00:38, 12.69s/it]

Epoch: 0 | Batch: 0 | Loss 0.6306712031364441:  50%|█████     | 2/4 [00:25<00:25, 12.72s/it]

Epoch: 0 | Batch: 1 | Loss 0.6791893839836121:  50%|█████     | 2/4 [00:25<00:25, 12.72s/it]

Epoch: 0 | Batch: 1 | Loss 0.6791893839836121:  75%|███████▌  | 3/4 [00:37<00:12, 12.60s/it]

Epoch: 0 | Batch: 2 | Loss 0.6786314249038696:  75%|███████▌  | 3/4 [00:37<00:12, 12.60s/it]

Epoch: 0 | Batch: 2 | Loss 0.6786314249038696: 100%|██████████| 4/4 [00:47<00:00, 11.34s/it]

Epoch: 0 | Batch: 3 | Loss 0.6368054747581482: 100%|██████████| 4/4 [00:47<00:00, 11.34s/it]

Epoch 0 Training Accuracy: tensor(0.6232)
Testing Accuracy: tensor(0.6129)




Epoch: 0 | Batch: 3 | Loss 0.6368054747581482: 5it [01:01, 12.52s/it]                       

Epoch: 1 | Batch: 0 | Loss 0.6148362755775452: 5it [01:01, 12.52s/it]

Epoch: 1 | Batch: 0 | Loss 0.6148362755775452: 6it [01:15, 12.87s/it]

Epoch: 1 | Batch: 1 | Loss 0.6576052308082581: 6it [01:15, 12.87s/it]

Epoch: 1 | Batch: 1 | Loss 0.6576052308082581: 7it [01:28, 12.97s/it]

Epoch: 1 | Batch: 2 | Loss 0.6245266199111938: 7it [01:28, 12.97s/it]

Epoch: 1 | Batch: 2 | Loss 0.6245266199111938: 8it [01:36, 11.26s/it]

Epoch: 1 | Batch: 3 | Loss 0.7673068046569824: 8it [01:36, 11.26s/it]

Epoch 1 Training Accuracy: tensor(0.6125)
Testing Accuracy: tensor(0.6129)




Epoch: 1 | Batch: 3 | Loss 0.7673068046569824: 9it [01:50, 12.28s/it]

Epoch: 2 | Batch: 0 | Loss 0.5730305910110474: 9it [01:50, 12.28s/it]

Epoch: 2 | Batch: 0 | Loss 0.5730305910110474: 10it [02:03, 12.57s/it]

Epoch: 2 | Batch: 1 | Loss 0.6400934457778931: 10it [02:03, 12.57s/it]

Epoch: 2 | Batch: 1 | Loss 0.6400934457778931: 11it [02:17, 12.76s/it]

Epoch: 2 | Batch: 2 | Loss 0.7136434316635132: 11it [02:17, 12.76s/it]

Epoch: 2 | Batch: 2 | Loss 0.7136434316635132: 12it [02:26, 11.83s/it]

Epoch: 2 | Batch: 3 | Loss 0.7074558138847351: 12it [02:26, 11.83s/it]

...

Epoch 298 Training Accuracy: tensor(1.)
Testing Accuracy: tensor(0.9355)




Epoch: 298 | Batch: 3 | Loss 0.0078068748116493225: 1197it [4:11:18, 12.67s/it]

Epoch: 299 | Batch: 0 | Loss 0.01249992847442627: 1197it [4:11:18, 12.67s/it]  

Epoch: 299 | Batch: 0 | Loss 0.01249992847442627: 1198it [4:11:31, 12.77s/it]

Epoch: 299 | Batch: 1 | Loss 0.01756807044148445: 1198it [4:11:31, 12.77s/it]

Epoch: 299 | Batch: 1 | Loss 0.01756807044148445: 1199it [4:11:45, 13.14s/it]

Epoch: 299 | Batch: 2 | Loss 0.017040347680449486: 1199it [4:11:45, 13.14s/it]

Epoch: 299 | Batch: 2 | Loss 0.017040347680449486: 1200it [4:11:53, 11.57s/it]

Epoch: 299 | Batch: 3 | Loss 0.0054497565142810345: 1200it [4:11:53, 11.57s/it]

Epoch 299 Training Accuracy: tensor(1.)


Epoch: 299 | Batch: 3 | Loss 0.0054497565142810345: 1200it [4:11:54, 12.60s/it]

Testing Accuracy: tensor(0.9355)


In [58]:
preds = []
labels = []
pbar = tqdm(total=len(rp_testloader), leave=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2.eval()
model_2.to(device)
for batch, (feature, label) in enumerate(rp_testloader):
  feature, label = feature.to(device), label.to(device)
  with torch.no_grad():
    predictions = model_2(feature)
    preds.append(predictions.cpu().numpy())
    labels.append(label.cpu().numpy())
  pbar.update()
  pbar.desc = f"Batch: {batch}"
  pbar.refresh()
pbar.close()
preds = torch.tensor(preds).ravel()
# preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
labels = torch.tensor(labels).ravel()
#acc = (preds == labels).sum() / len(preds)
acc =binary_accuracy(preds, labels)
print(acc)

Batch: 0: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

tensor(0.9355)
